# 1. 스키마 설계

In [9]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='gmarketbest', charset='utf8')
cursor = db.cursor()

sql = '''
create table items (
    item_code varchar(100) not null primary key,
    title varchar(200) not null,
    ori_price int not null,
    dis_price int not null,
    discount_percent int not null,
    provider varchar(50)
);
'''

cursor.execute(sql)

sql ='''
create table ranking (
    num int not null auto_increment primary key,
    main_category varchar(50) not null,
    sub_category varchar(50) not null,
    item_ranking int not null,
    item_code varchar(100) not null,
    foreign key(item_code) references items(item_code)    
);
'''
cursor.execute(sql)

db.commit()
db.close()

# 2. 크롤링 

####  2-1. 메인 카테고리 정보 가져오기

In [17]:
import requests
from bs4 import BeautifulSoup

res = requests.get("http://corners.gmarket.co.kr/Bestsellers")
soup = BeautifulSoup(res.content, 'html.parser')
main_categories = soup.select("div.gbest-cate ul.by-group li a")
for main_category in main_categories:
    print('http://corners.gmarket.co.kr'+main_category['href'], main_category.get_text())

http://corners.gmarket.co.kr/Bestsellers ALL
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G01 패션의류
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G02 신발/잡화
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G03 화장품/헤어
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G04 유아동/출산
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G07 식품
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G08 생활/주방/건강
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G09 가구/침구
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G05 스포츠/자동차
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G06 컴퓨터/전자
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G10 도서/음반
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G11 여행
http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G12 e쿠폰/티켓


#### 2-2. 메인, 서브 카테고리 + 상품정보 가져오기

In [15]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='watson1259@', db='gmarketbest', charset='utf8')
cursor = db.cursor()

res = requests.get("http://corners.gmarket.co.kr/Bestsellers")
soup = BeautifulSoup(res.content, 'html.parser')
main_categories = soup.select("div.gbest-cate ul.by-group li a")
for main_category in main_categories:
    get_sub_category('http://corners.gmarket.co.kr'+main_category['href'], main_category.get_text())

http://corners.gmarket.co.kr/Bestsellers ALL
INSERT INTO items values('1759110607','[아웃팅]3D 마스크 10매+정전기필터 10매 일회용 마스크',26000,
        16900,35,'아웃팅21')
INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'all','1','1759110607')
INSERT INTO items values('751246244','쏭스타일 신상추가.맨투맨.루즈핏.롱티',43000,
        12900,70,'ssongstyle')
INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'all','2','751246244')
INSERT INTO items values('1558248357','[니베아]니베아 맨 딥 쉐이빙폼 / 면도크림 X 3개 200ml',13900,
        9900,28,'바이어스도르프 공식')
INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'all','3','1558248357')
INSERT INTO items values('1759113133','[아웃팅]정전기필터 마스크 필터 50매 한박스 일회용마스크',29900,
        29900,0,'아웃팅21')
INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('ALL',
    'all','4','1759113133')
INSERT INTO items values('1759503236','퓨어365 황사방역마스크 KF94 대형 5매입(화이트)

KeyboardInterrupt: 

In [14]:
def get_sub_category(main_category_link, main_category_name):
    print(main_category_link, main_category_name)
    res = requests.get(main_category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, main_category_name, 'all')
    
    sub_categories = soup.select("div.navi.group ul li a")
    for sub_category in sub_categories:
        res = requests.get(sub_category)
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, main_category_name, sub_category.get_text())

In [13]:
def get_items(html, main_category_name, sub_category_name):
    items_result_list = list()
    results = html.select('div.best-list')
    for index,result in enumerate(results[1].select('li')):
        data_dict = dict()
        
        ranking = index + 1
        title = result.select_one('a.itemname').get_text()
        ori_price = result.select_one('div.o-price') 
        dis_price = result.select_one('div.s-price strong span')
        if ori_price is None or ori_price.get_text() =='':
            ori_price = dis_price
        if dis_price is None:
            ori_price, dis_price = 0, 0
        else:
            ori_price = ori_price.get_text().replace('원','').replace(',','')
            dis_price = dis_price.get_text().replace('원','').replace(',','')
        dis_rate = result.select_one('div.s-price span em')
        if dis_rate is None or dis_rate.get_text() == '':
            dis_rate = 0
        else:
            dis_rate = dis_rate.get_text().replace('%','')
        
        product_link = result.select_one('div.thumb > a')
        item_code_l = product_link.attrs['href'].split('=')[1]
        item_code = item_code_l.split('&')[0]
        
        res = requests.get(product_link.attrs['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one("div.item-topinfo_headline > p > a > strong")
        if provider is None:
            provider = ''
        else:
            provider = provider.get_text()
        
        data_dict['main_category_name'] = main_category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider
        data_dict['title'] = title
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['dis_rate'] = dis_rate
        save_data(data_dict)
        
       
        #print(main_category_name, sub_category_name, ranking, item_code, provider, title, ori_price, dis_price, dis_rate)

#### 2-3. 크롤링한 데이터 MySQL DB에 담기

In [12]:
def save_data(item_info):
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '"""+item_info['item_code']+"""';"""
    cursor.execute(sql)
    result = cursor.fetchone()
    if result[0] == 0:
        sql = """INSERT INTO items values('"""+item_info['item_code']+"""','"""+item_info['title']+"""',"""+str(item_info['ori_price'])+""",
        """+str(item_info['dis_price'])+""","""+str(item_info['dis_rate'])+""",'"""+item_info['provider']+"""')"""
        print(sql)
        cursor.execute(sql)
        
    sql = """INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('"""+item_info['main_category_name']+"""',
    '"""+item_info['sub_category_name']+"""','"""+str(item_info['ranking'])+"""','"""+item_info['item_code']+"""')"""
    print(sql)
    cursor.execute(sql)

#### cf) Insert 구문 test

In [1]:

item_info = {'main_category_name': 'ALL', 'sub_category_name': 'all', 'ranking': 1, 'item_code': '939528083', 'provider': '한예지공식샵', 'title': '한예지 볼륨앤소프트 천연펄프 3겹  화장지 30롤 3팩', 'ori_price': '50000', 'dis_price': '20900', 'dis_rate': '58'}

In [2]:
sql = """
INSERT INTO items values('
"""+item_info['item_code']+"""',
'"""+item_info['title']+"""',
"""+item_info['ori_price']+""",
"""+item_info['dis_price']+""",
"""+item_info['dis_rate']+""",
'"""+item_info['provider']+"""')
"""
    
print(sql.replace('\n',''))

INSERT INTO items values('939528083','한예지 볼륨앤소프트 천연펄프 3겹  화장지 30롤 3팩',50000,20900,58,'한예지공식샵')


In [ ]:
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);

In [3]:
sql = """
INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('
"""+item_info['main_category_name']+"""',
'"""+item_info['sub_category_name']+"""',
'"""+str(item_info['ranking'])+"""',
'"""+item_info['item_code']+"""')
"""
print(sql.replace('\n',''))

INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('ALL','all','1','939528083')
